In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("UserProfileEDA").getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1746553043698_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Step 2: Load users.tsv from S3
df = spark.read.option("header", True).option("sep", "\t").csv("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/users.tsv")

# Step 3: Drop unwanted columns
df = df.drop("WindowID", "Split", "GraduationDate")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql.functions import col, isnan, when, count

df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c + "_nulls") for c in df.columns]).show()

# Step 7: Value counts for categorical columns
categorical_cols = ["City", "State", "Country", "DegreeType", "Major"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+----------+-----------+-------------+-------------+----------------+-----------+----------------------+--------------------------+-----------------------+-------------------+--------------------+
|UserID_nulls|City_nulls|State_nulls|Country_nulls|ZipCode_nulls|DegreeType_nulls|Major_nulls|WorkHistoryCount_nulls|TotalYearsExperience_nulls|CurrentlyEmployed_nulls|ManagedOthers_nulls|ManagedHowMany_nulls|
+------------+----------+-----------+-------------+-------------+----------------+-----------+----------------------+--------------------------+-----------------------+-------------------+--------------------+
|           0|         0|        490|            0|         1734|               0|      96897|                     0|                     14180|                  42076|                  0|                   0|
+------------+----------+-----------+-------------+-------------+----------------+-----------+----------------------+--------------------------+----------------

In [4]:
for col_name in categorical_cols:
    print(f"--- Value counts for {col_name} ---")
    df.groupBy(col_name).count().orderBy("count", ascending=False).show(10)

# Step 8: Summary statistics for numerical columns
numeric_cols = ["WorkHistoryCount", "TotalYearsExperience"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--- Value counts for City ---
+------------+-----+
|        City|count|
+------------+-----+
|     Chicago| 7675|
|     Houston| 6013|
|Indianapolis| 4871|
|       Miami| 4750|
|Philadelphia| 4710|
|   Charlotte| 4108|
|     Orlando| 3898|
|      Dallas| 3898|
|     Phoenix| 3647|
| Kansas City| 2947|
+------------+-----+
only showing top 10 rows

--- Value counts for State ---
+-----+-----+
|State|count|
+-----+-----+
|   FL|43108|
|   TX|35874|
|   CA|33026|
|   IL|24323|
|   NY|20590|
|   PA|17107|
|   GA|16162|
|   NJ|15709|
|   OH|15499|
|   NC|14756|
+-----+-----+
only showing top 10 rows

--- Value counts for Country ---
+-------+------+
|Country| count|
+-------+------+
|     US|388499|
|     IN|   258|
|     CA|   113|
|     PK|    65|
|     PH|    60|
|     MX|    50|
|     UK|    44|
|     ES|    42|
|     FR|    32|
|     IT|    29|
+-------+------+
only showing top 10 rows

--- Value counts for DegreeType ---
+-----------+------+
| DegreeType| count|
+-----------+------+
|

In [8]:
df.select([col(c).cast("float") for c in numeric_cols]).describe().show()

# Step 9: Unique user count
print(f"Total distinct users: {df.select('UserID').distinct().count()}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------------------+
|summary|  WorkHistoryCount|TotalYearsExperience|
+-------+------------------+--------------------+
|  count|            389708|              375528|
|   mean| 4.500238640212672|  12.325898468289981|
| stddev|2.3563660902089287|   8.726533475048155|
|    min|               0.0|                 0.0|
|    max|             120.0|               112.0|
+-------+------------------+--------------------+

Total distinct users: 389708

In [11]:
df = df.select(
    "UserID", "WorkHistoryCount", "TotalYearsExperience"
).withColumn("WorkHistoryCount", col("WorkHistoryCount").cast("float")) \
 .withColumn("TotalYearsExperience", col("TotalYearsExperience").cast("float"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
indexers = [
    StringIndexer(inputCol="State", outputCol="State_indexed", handleInvalid='keep'),
    StringIndexer(inputCol="Major", outputCol="Major_indexed", handleInvalid='keep')
]

for indexer in indexers:
    df = indexer.fit(df).transform(users_df)

In [5]:
df = df.drop("State", "Major")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Step 6: Load user history embeddings (assuming this is already saved as Parquet or CSV)
user_history_df = spark.read.parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/user_job_history_embeddings.parquet")
# This DF is expected to have: UserID, job_title_embedding (Vector)

# Step 7: Join on UserID
final_user_df = df.join(user_history_df, on="UserID", how="inner")

# Step 8: Preview joined data
final_user_df.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
final_user_df.show(1, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [14]:
# Save the final DataFrame to S3 in Parquet format
final_user_df.write.mode("overwrite").parquet("s3://aws-emr-studio-201107706822-us-east-1/1746053900403/e-6HYCE3WANA8IROUS7NBVDSOU4/processed/final_user_data/")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…